In [1]:
import pandas as pd

In [4]:
# Load SemEval train and test datasets
semeval_train = pd.read_csv("/s/babbage/b/nobackup/nblancha/public-datasets/ani/MUStARD/MUStARD/SEMeval/iSarcasmEval/train/train.En.csv")  # Update with correct file path
semeval_test = pd.read_csv("/s/babbage/b/nobackup/nblancha/public-datasets/ani/MUStARD/MUStARD/SEMeval/iSarcasmEval/test/task_A_En_test.csv")    # Update with correct file path

In [7]:
semeval_test

,text,sarcastic
0,"Size on the the Toulouse team, That pack is mo...",0
1,Pinball!,0
2,So the Scottish Government want people to get ...,1
3,villainous pro tip : change the device name on...,0
4,I would date any of these men 🥺,0
...,...,...
1395,I’ve just seen this and felt it deserved a Ret...,0
1396,Omg how an earth is that a pen !!! 🤡,0
1397,Bringing Kanye and drake to a tl near you,0
1398,"I love it when women are referred to as ""girl ...",1


In [8]:
# Extract relevant columns
semeval_train = semeval_train[["tweet", "sarcastic"]]
semeval_test = semeval_test[["text", "sarcastic"]]

In [9]:
# Rename columns for consistency with MUStARD
semeval_train.columns = ["utterance", "label"]
semeval_test.columns = ["utterance", "label"]

In [10]:
# Convert labels to 0 (Not Sarcastic) and 1 (Sarcastic)
semeval_train["label"] = semeval_train["label"].astype(int)
semeval_test["label"] = semeval_test["label"].astype(int)

/tmp/ipykernel_679787/4092932509.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  semeval_train["label"] = semeval_train["label"].astype(int)


In [11]:
# Save preprocessed SemEval dataset
semeval_train.to_csv("SemEval_train_processed.csv", index=False)
semeval_test.to_csv("SemEval_test_processed.csv", index=False)

In [12]:
print(f"✅ SemEval Preprocessing Done: {len(semeval_train)} Train, {len(semeval_test)} Test Samples")
print("✅ Preprocessed data saved as 'SemEval_train_processed.csv' and 'SemEval_test_processed.csv'")

✅ SemEval Preprocessing Done: 3468 Train, 1400 Test Samples
✅ Preprocessed data saved as 'SemEval_train_processed.csv' and 'SemEval_test_processed.csv'


In [15]:
import pandas as pd
import torch
from transformers import BertTokenizer

# Load preprocessed SemEval datasets
semeval_train = pd.read_csv("SemEval_train_processed.csv")
semeval_test = pd.read_csv("SemEval_test_processed.csv")

# Ensure 'utterance' column has no NaN values and is in string format
semeval_train["utterance"] = semeval_train["utterance"].fillna("").astype(str)
semeval_test["utterance"] = semeval_test["utterance"].fillna("").astype(str)

# Load BERT tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Tokenize training and test data
semeval_train_encodings = tokenizer(
    semeval_train["utterance"].tolist(), truncation=True, padding=True, max_length=128, return_tensors="pt"
)
semeval_test_encodings = tokenizer(
    semeval_test["utterance"].tolist(), truncation=True, padding=True, max_length=128, return_tensors="pt"
)

# Convert labels to tensor format
semeval_train_labels = torch.tensor(semeval_train["label"].tolist())
semeval_test_labels = torch.tensor(semeval_test["label"].tolist())

# Save tokenized data
torch.save({"input_ids": semeval_train_encodings["input_ids"], "attention_mask": semeval_train_encodings["attention_mask"], "labels": semeval_train_labels}, "SemEval_train.pt")
torch.save({"input_ids": semeval_test_encodings["input_ids"], "attention_mask": semeval_test_encodings["attention_mask"], "labels": semeval_test_labels}, "SemEval_test.pt")

print(f"✅ SemEval Tokenization Complete: {len(semeval_train_labels)} Train, {len(semeval_test_labels)} Test Samples")
print("✅ Tokenized data saved as 'SemEval_train.pt' and 'SemEval_test.pt'.")


✅ SemEval Tokenization Complete: 3468 Train, 1400 Test Samples
✅ Tokenized data saved as 'SemEval_train.pt' and 'SemEval_test.pt'.


In [16]:
import numpy as np
import torch
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from transformers import BertForSequenceClassification

# Load tokenized SemEval test data
semeval_test_data = torch.load("SemEval_test.pt")

# Load trained BERT model (trained on MUStARD)
model = BertForSequenceClassification.from_pretrained("./sarcasm_bert_model")
model.eval()  # Set model to evaluation mode

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Prepare SemEval test data
input_ids = semeval_test_data["input_ids"].to(device)
attention_mask = semeval_test_data["attention_mask"].to(device)
labels = semeval_test_data["labels"].to(device)

# Run inference
with torch.no_grad():
    outputs = model(input_ids, attention_mask=attention_mask)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)

# Convert predictions and labels to numpy arrays
preds = predictions.cpu().numpy()
true_labels = labels.cpu().numpy()

# Compute Evaluation Metrics
accuracy = accuracy_score(true_labels, preds)
f1 = f1_score(true_labels, preds)
precision = precision_score(true_labels, preds)
recall = recall_score(true_labels, preds)

# Print and Save Results
print(f"✅ Evaluation on SemEval Complete!")
print(f"📊 Accuracy: {accuracy:.4f}")
print(f"📊 F1-Score: {f1:.4f}")
print(f"📊 Precision: {precision:.4f}")
print(f"📊 Recall: {recall:.4f}")

# Save results to a file
results = {
    "accuracy": accuracy,
    "f1_score": f1,
    "precision": precision,
    "recall": recall
}
np.save("bert_semeval_results.npy", results)
print("✅ Evaluation results saved to 'bert_semeval_results.npy'.")


/s/babbage/b/nobackup/nblancha/merry/conda/envs/ani/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/s/babbage/b/nobackup/nblancha/merry/conda/envs/ani/lib/python3.8/site-packages/torchvision/io/image.py:11: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")
2025-02-08 16:28:03.539548: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-08 16:28:03.580007: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with t

✅ Evaluation on SemEval Complete!
📊 Accuracy: 0.2914
📊 F1-Score: 0.2298
📊 Precision: 0.1360
📊 Recall: 0.7400
✅ Evaluation results saved to 'bert_semeval_results.npy'.
